In [8]:
# Include necessary imports
import os
import torch 
import pandas as pd
from torch.utils.data import DataLoader
from music21 import *
import numpy as np

In [9]:
# Preprocess the data

folder_path = 'Data/'
test = []
train = []
validation = []
for dirname in os.listdir(folder_path):
    if dirname != '.DS_Store':
        for filename in os.listdir(folder_path + dirname):
            df = pd.read_csv(folder_path + dirname + '/' + filename)
            transposed_df = df.transpose()
            if dirname == 'test':
                test.append(transposed_df)
            if dirname == 'train':
                train.append(transposed_df)
            if dirname == 'valid':
                validation.append(transposed_df)

# Model

In [10]:
class Model(torch.nn.Module):
    def __init__(self, input_size, output_size, hidden_dim=50, n_layers=1):
        super(Model, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.lstm = torch.nn.LSTM(input_size, hidden_dim, n_layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, output_size) # fully connected layer
        
    def forward(self, x, hidden=None):
        lstm_output, (h,c) = self.lstm(x, hidden)
        model_output = self.fc(lstm_output)
        return model_output

# Train

In [11]:
def train_model(model, melody, harmonies, optimizer, criterion, num_epochs):
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        output = model(melody)
        loss = criterion(output, harmonies)
        loss.backward()
        optimizer.step()
        if (epoch + 1) % 100 == 0:
            print("Epoch: ", epoch, "Loss: ", loss)

In [12]:
criterion = torch.nn.MSELoss()
# for song in train:
melody = torch.tensor(train[0].iloc[0], dtype=torch.float32).unsqueeze(0).reshape(1,train[0].shape[1],1)
harmonies = torch.transpose(torch.tensor(train[0].iloc[1:].values, dtype=torch.float32),0,1).unsqueeze(0)
model = Model(1, harmonies.shape[2])
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
train_model(model, melody, harmonies, optimizer, criterion, 10000)

Epoch:  99 Loss:  tensor(529.5349, grad_fn=<MseLossBackward0>)
Epoch:  199 Loss:  tensor(42.1777, grad_fn=<MseLossBackward0>)
Epoch:  299 Loss:  tensor(13.3016, grad_fn=<MseLossBackward0>)
Epoch:  399 Loss:  tensor(12.5561, grad_fn=<MseLossBackward0>)
Epoch:  499 Loss:  tensor(12.5471, grad_fn=<MseLossBackward0>)
Epoch:  599 Loss:  tensor(12.5463, grad_fn=<MseLossBackward0>)
Epoch:  699 Loss:  tensor(12.5456, grad_fn=<MseLossBackward0>)
Epoch:  799 Loss:  tensor(12.5450, grad_fn=<MseLossBackward0>)
Epoch:  899 Loss:  tensor(12.5446, grad_fn=<MseLossBackward0>)
Epoch:  999 Loss:  tensor(12.5442, grad_fn=<MseLossBackward0>)
Epoch:  1099 Loss:  tensor(12.5438, grad_fn=<MseLossBackward0>)
Epoch:  1199 Loss:  tensor(12.5434, grad_fn=<MseLossBackward0>)
Epoch:  1299 Loss:  tensor(12.5431, grad_fn=<MseLossBackward0>)
Epoch:  1399 Loss:  tensor(12.5428, grad_fn=<MseLossBackward0>)
Epoch:  1499 Loss:  tensor(12.5425, grad_fn=<MseLossBackward0>)
Epoch:  1599 Loss:  tensor(12.5422, grad_fn=<MseLo

In [15]:
# tiny = pd.DataFrame([[67,62,59,43], [68,62,59,43]]).transpose()
# melody = torch.tensor(tiny.iloc[0], dtype=torch.float32).unsqueeze(0).reshape(1,2,1)
# harmonies = torch.transpose(torch.tensor(tiny.iloc[1:].values, dtype=torch.float32),0,1).unsqueeze(0)
# model = Model(1, harmonies.shape[2])
# optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
# criterion = torch.nn.MSELoss()
# train_model(model, melody, harmonies, optimizer, criterion, 10000)

In [28]:
melody = train[0].iloc[0]
result = model(torch.tensor(melody, dtype=torch.float32).unsqueeze(0).reshape(1, train[0].shape[1], 1))

score = stream.Score()
melody_part = stream.Part()
alto_part = stream.Part()
tenor_part = stream.Part()
bass_part = stream.Part()

for pitch in melody:
    melody_note = note.Note(int(pitch))
    melody_part.append(melody_note)

alto_notes = result[0, :, 0]
tenor_notes = result[0, :, 1]
bass_notes = result[0, :, 2]  

for pitch in alto_notes:
    alto_note = note.Note(int(pitch.item()))
    alto_part.append(alto_note)
for pitch in tenor_notes:
     tenor_note = note.Note(int(pitch.item()))
     tenor_part.append(tenor_note)
for pitch in bass_notes:
    bass_note = note.Note(int(pitch.item()))
    bass_part.append(bass_note)

score.append(melody_part)
score.append(alto_part)
score.append(tenor_part)
score.append(bass_part)
score.show('midi')
score.write('musicxml', 'output.xml')

WindowsPath('C:/Users/foodrunner/CS370/PolyphAI/Code/output.xml')

In [ ]:
# Finetune (hyperparameters, move around test data (refer to notes), etc)

In [ ]:
# Test with new data + evaluate

In [ ]:
# Make any other changes

In [ ]:
# Sheet music + audio (musicAI)

In [ ]:
# Create new models if time permits (follow steps 3 - 7)

In [ ]:
# Compare models

In [ ]:
# Front end ** if time permits
# - Interactive sheet music
# - musescore front end??